In [ ]:
# from data.marco import get_sentences
# from model.word2vec import get_model, train_model
# from gensim.utils import simple_preprocess

# raw_sentences = get_sentences()
# tokenized_sentences = [simple_preprocess(sentence) for sentence in raw_sentences]
# skipgram_model = get_model(tokenized_sentences)
# train_model(skipgram_model, tokenized_sentences)

# model_path = "./artifacts/word2vec-300.bin"
# skipgram_model.save(model_path)

In [1]:
from gensim.models import Word2Vec

model_path = "./artifacts/word2vec-300.bin"
model = Word2Vec.load(model_path)
model.wv['what']

array([-0.08166109, -0.05603082,  0.08131558,  0.1453118 , -0.02772139,
        0.3336876 ,  0.26576355, -0.20260681,  0.18647544, -0.06265885,
       -0.02537769,  0.27152473,  0.18361786, -0.22557825,  0.06676365,
       -0.14489435,  0.20553292,  0.03955898,  0.21805419,  0.32962334,
        0.03388114,  0.21090184,  0.20374602, -0.11790657, -0.10713933,
       -0.17888841,  0.12195691,  0.08326802,  0.08338279,  0.1702925 ,
        0.42584544, -0.15946382,  0.00752363,  0.36911058,  0.00319007,
        0.09264715, -0.07176673, -0.14241076, -0.04268489, -0.10397603,
        0.3095556 ,  0.09982298,  0.12000576, -0.0958479 ,  0.0315978 ,
       -0.29556945,  0.1510915 ,  0.21604806,  0.14024083, -0.32941374,
        0.27504742,  0.15409113,  0.0196512 , -0.1924288 , -0.13030423,
        0.28063247,  0.0216731 ,  0.01504864,  0.09892328, -0.09026737,
       -0.07826483,  0.08209728,  0.00135547, -0.12017546,  0.224457  ,
       -0.2233952 ,  0.20976582, -0.02407778,  0.17635925, -0.21

In [2]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate(batch):
    # Unzip the batch into lists of queries, relevant_docs, and irrelevant_docs
    query_emb_list, relevant_doc_emb_list, irrelevant_doc_emb_list = zip(*batch)
    
    # Pad sequences for relevant and irrelevant documents
    # This assumes each document is already a tensor of embeddings; adjust if the structure is different
    padded_relevant = pad_sequence([pad_sequence(docs, batch_first=True) for docs in relevant_doc_emb_list], batch_first=True)
    padded_irrelevant = pad_sequence([pad_sequence(docs, batch_first=True) for docs in irrelevant_doc_emb_list], batch_first=True)
    
    # Pad the sequences of query embeddings
    query_emb_tensor = pad_sequence(query_emb_list, batch_first=True)

    return query_emb_tensor, padded_relevant, padded_irrelevant

In [ ]:
from model.twotowermodel import DocumentTower, QueryTower
from data.dataset import QueryDocumentDataset
from data.marco import get_training_dataset
from torch.optim import Adam
from torch.utils.data import DataLoader
from utils.loss_function import triplet_loss_function
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model and optimizer
document_model = DocumentTower(embedding_dim=300, hidden_dim=128).to(device)
query_model = QueryTower(embedding_dim=300, hidden_dim=128).to(device)

optimizer = Adam(list(document_model.parameters()) + list(query_model.parameters()), lr=0.001)

dataset_instance = QueryDocumentDataset(data=get_training_dataset(), embedding_model=model)
dataloader = DataLoader(dataset_instance, batch_size=32, shuffle=False, collate_fn=custom_collate)  # You can adjust batch_size as needed

counter = 0

for query_emb, relevant_doc_emb, irrelevant_doc_emb in dataloader:
    # Convert the tokens to embeddings
    # Forward pass to get encodings for two tower

    query_emb = query_emb.to(device)
    relevant_doc_emb = relevant_doc_emb.to(device)
    irrelevant_doc_emb = irrelevant_doc_emb.to(device)

    relevant_doc_encoding, irrelevant_doc_encoding = document_model(relevant_doc_emb, irrelevant_doc_emb)
    query_encoding = query_model(query_emb)

    # Compute triplet loss
    loss = triplet_loss_function(query_encoding, relevant_doc_encoding, irrelevant_doc_encoding, margin=0.3)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    counter += 1

    if counter % 1000 == 0:
        print(f"Triplet loss: {loss.item()}")
